In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from pathlib import Path
import requests
import s3fs
from os.path import dirname, join
from scipy.interpolate import RegularGridInterpolator

# Earthdata
from earthdata import DataCollections, DataGranules, Auth, Store

# Define paths and directories

In [2]:
insitu_dir=Path('/home/jovyan/Data/SASSIE/collocation/insitu')  
output_dir=Path('/home/jovyan/Data/SASSIE/collocation/satellite/smap_jpl_l3') 

# Define important parameters

In [12]:
# beaufort sea region
lonrange = [-160, -130]
latrange = [68, 80]
bounding_box = (lonrange[0], latrange[0], lonrange[1], latrange[1])
# Data collection short name
ShortName_sss = "SMAP_JPL_L3_SSS_CAP_8DAY-RUNNINGMEAN_V5"

In [4]:
# See: https://github.com/nsidc/earthdata
CollectionQuery = DataCollections().short_name(ShortName_sss).cloud_hosted(True)
collections = CollectionQuery.get()

for collection in collections:
    concept_id = collection.concept_id()
    print(concept_id)

C2208422957-POCLOUD


In [31]:
# # Cloud:
# granule_url = 'https://cmr.earthdata.nasa.gov/search/granules'

# #SMAP JPL earthdata info from earthdatasearch
# ShortName_sss = "SMAP_JPL_L3_SSS_CAP_8DAY-RUNNINGMEAN_V5" #can't open this one, the url has https: instead of s3:
# # ShortName_sss = "OISSS_L4_multimission_7day_v1"
# # ShortName_sss = "ECCO_L4_SSH_05DEG_DAILY_V4R4B"

# #OISST earthdata info
# # concept_id_sst = 

# # Ask PODAAC for the collection id
# response = requests.get(
#     url='https://cmr.earthdata.nasa.gov/search/collections.umm_json', 
#     params={'provider': "POCLOUD",
#             'ShortName': ShortName_sss,
#             'page_size': 1}
# )
# ummc = response.json()['items'][0]
# concept_id_sss = ummc['meta']['concept-id']
# print(f'collection id: {concept_id_sss}')

# Cloud Credentials

In [5]:
auth = Auth().login(strategy='netrc')
if auth.authenticated is None:
    # we ask for credentials
    auth.login()

You're now authenticated with NASA Earthdata Login


In [6]:
# IMPORTANT: This session will be valid for 1 hour
fs = Store(auth).get_s3fs_session('POCLOUD')
fs

# Load in situ SIZRS file for 1 year

In [7]:
filename=str(insitu_dir) + '/SIZRS_2016.nc'
insitu = xr.open_dataset(filename)
insitu

<xarray.Dataset>
Dimensions:  (t: 26)
Coordinates:
    x        (t) float64 ...
    y        (t) float64 ...
    z        float64 ...
  * t        (t) datetime64[ns] 2016-06-15T20:16:00 ... 2016-10-06T22:39:00
Data variables:
    SSS      (t) float64 ...
    SST      (t) float64 ...

In [8]:
# round the time to the closest day
time = pd.to_datetime(insitu.t).round('D')
# convert again to datetime64
time=time.values
time

array(['2016-06-16T00:00:00.000000000', '2016-06-16T00:00:00.000000000',
       '2016-06-16T00:00:00.000000000', '2016-06-16T00:00:00.000000000',
       '2016-07-14T00:00:00.000000000', '2016-07-14T00:00:00.000000000',
       '2016-07-14T00:00:00.000000000', '2016-07-14T00:00:00.000000000',
       '2016-07-14T00:00:00.000000000', '2016-08-19T00:00:00.000000000',
       '2016-08-19T00:00:00.000000000', '2016-09-15T00:00:00.000000000',
       '2016-09-15T00:00:00.000000000', '2016-09-15T00:00:00.000000000',
       '2016-09-15T00:00:00.000000000', '2016-09-15T00:00:00.000000000',
       '2016-09-15T00:00:00.000000000', '2016-09-16T00:00:00.000000000',
       '2016-09-16T00:00:00.000000000', '2016-09-16T00:00:00.000000000',
       '2016-10-07T00:00:00.000000000', '2016-10-07T00:00:00.000000000',
       '2016-10-07T00:00:00.000000000', '2016-10-07T00:00:00.000000000',
       '2016-10-07T00:00:00.000000000', '2016-10-07T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [22]:
date_ranges = []
for t in time:
    start_date = str(t)
    end_date = str(t+np.timedelta64(59+59*60+23*60*60, 's'))
    date_range = (start_date, end_date) # this format is needed for 'requests'
    date_ranges.append(date_range)
date_ranges

[('2016-06-16T00:00:00.000000000', '2016-06-16T23:59:59.000000000'),
 ('2016-06-16T00:00:00.000000000', '2016-06-16T23:59:59.000000000'),
 ('2016-06-16T00:00:00.000000000', '2016-06-16T23:59:59.000000000'),
 ('2016-06-16T00:00:00.000000000', '2016-06-16T23:59:59.000000000'),
 ('2016-07-14T00:00:00.000000000', '2016-07-14T23:59:59.000000000'),
 ('2016-07-14T00:00:00.000000000', '2016-07-14T23:59:59.000000000'),
 ('2016-07-14T00:00:00.000000000', '2016-07-14T23:59:59.000000000'),
 ('2016-07-14T00:00:00.000000000', '2016-07-14T23:59:59.000000000'),
 ('2016-07-14T00:00:00.000000000', '2016-07-14T23:59:59.000000000'),
 ('2016-08-19T00:00:00.000000000', '2016-08-19T23:59:59.000000000'),
 ('2016-08-19T00:00:00.000000000', '2016-08-19T23:59:59.000000000'),
 ('2016-09-15T00:00:00.000000000', '2016-09-15T23:59:59.000000000'),
 ('2016-09-15T00:00:00.000000000', '2016-09-15T23:59:59.000000000'),
 ('2016-09-15T00:00:00.000000000', '2016-09-15T23:59:59.000000000'),
 ('2016-09-15T00:00:00.000000000',

In [24]:
matching_granules = [] #list of granules to load
for dt in date_ranges:
    download_size = 0
    GranuleQuery = DataGranules().parameters(
        concept_id=concept_id,
        bounding_box=bounding_box,
        temporal=dt)
    granules = GranuleQuery.get()
    for granule in granules:
        # we use extend to aggregate the results in the same list.
        # to get the s3 links: matching_granules.extend(granule.data_links(s3_only=True))
        matching_granules.extend(granule.data_links(s3_only=True))
        download_size += granule.size()
        
        
        # url = granule['links'][0]['href']
        # id = granule['title'] + '.nc'
        # r = requests.get(url)
        # out_file = Path(output_dir, id)
        # granule_time=str(np.datetime64(granule['time_start'])+np.timedelta64(4, 'D'))
        # # only download the granule if granule time + 4 days is between start_date and end_date
        # if (granule_time >= start_date) & (granule_time <= end_date):
        #     print(url)
        #     print("date range: "+str(start_date)+'-'+str(end_date)+', file: '+str(id))
        #     granules_load.append(f'{output_dir}/{id}')
        #     # if the file doesn't exist locally, download
        #     if (not out_file.exists()):
        #         print('downloading', url, id)
        #         with open(out_file, 'wb') as f:
        #             f.write(r.content)    
        #     else:
        #         print('file already downloaded')  
    
    print(f"date: {dt} , total granules: {len(granules)}, size(MB): {round(download_size, 2)}")
matching_granules

date: ('2016-06-16T00:00:00.000000000', '2016-06-16T23:59:59.000000000') , total granules: 9, size(MB): 89.04
date: ('2016-06-16T00:00:00.000000000', '2016-06-16T23:59:59.000000000') , total granules: 9, size(MB): 89.04
date: ('2016-06-16T00:00:00.000000000', '2016-06-16T23:59:59.000000000') , total granules: 9, size(MB): 89.04
date: ('2016-06-16T00:00:00.000000000', '2016-06-16T23:59:59.000000000') , total granules: 9, size(MB): 89.04
date: ('2016-07-14T00:00:00.000000000', '2016-07-14T23:59:59.000000000') , total granules: 9, size(MB): 94.85
date: ('2016-07-14T00:00:00.000000000', '2016-07-14T23:59:59.000000000') , total granules: 9, size(MB): 94.85
date: ('2016-07-14T00:00:00.000000000', '2016-07-14T23:59:59.000000000') , total granules: 9, size(MB): 94.85
date: ('2016-07-14T00:00:00.000000000', '2016-07-14T23:59:59.000000000') , total granules: 9, size(MB): 94.85
date: ('2016-07-14T00:00:00.000000000', '2016-07-14T23:59:59.000000000') , total granules: 9, size(MB): 94.85
date: ('20

['https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/SMAP_JPL_L3_SSS_CAP_8DAY-RUNNINGMEAN_V5/2016/160/SMAP_L3_SSS_20160612_8DAYS_V5.0.nc',
 's3://podaac-ops-cumulus-protected/SMAP_JPL_L3_SSS_CAP_8DAY-RUNNINGMEAN_V5/2016/160/SMAP_L3_SSS_20160612_8DAYS_V5.0.nc',
 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/SMAP_JPL_L3_SSS_CAP_8DAY-RUNNINGMEAN_V5/2016/161/SMAP_L3_SSS_20160613_8DAYS_V5.0.nc',
 's3://podaac-ops-cumulus-protected/SMAP_JPL_L3_SSS_CAP_8DAY-RUNNINGMEAN_V5/2016/161/SMAP_L3_SSS_20160613_8DAYS_V5.0.nc',
 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/SMAP_JPL_L3_SSS_CAP_8DAY-RUNNINGMEAN_V5/2016/162/SMAP_L3_SSS_20160614_8DAYS_V5.0.nc',
 's3://podaac-ops-cumulus-protected/SMAP_JPL_L3_SSS_CAP_8DAY-RUNNINGMEAN_V5/2016/162/SMAP_L3_SSS_20160614_8DAYS_V5.0.nc',
 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/SMAP_JPL_L3_SSS_CAP_8DAY-RUNNINGMEAN_V5/2016/163/SMAP_L3_SSS_20160615_8DAYS_V5.0.nc'

In [47]:
vars(granule)['_dict']['umm']['BeginningDateTime']

KeyError: 'BeginningDateTime'

In [42]:
vars(granule)

{'_keypath_separator': '.',
 '_dict': {'meta': {'concept-type': 'granule',
   'concept-id': 'G2215604518-POCLOUD',
   'revision-id': 1,
   'native-id': 'SMAP_L3_SSS_20161011_8DAYS_V5.0',
   'provider-id': 'POCLOUD',
   'format': 'application/vnd.nasa.cmr.umm+json',
   'revision-date': '2022-02-11T18:21:55.389Z'},
  'umm': {'TemporalExtent': {'RangeDateTime': {'EndingDateTime': '2016-10-15T12:00:00.000Z',
     'BeginningDateTime': '2016-10-07T12:00:00.000Z'}},
   'MetadataSpecification': {'URL': 'https://cdn.earthdata.nasa.gov/umm/granule/v1.6.4',
    'Name': 'UMM-G',
    'Version': '1.6.4'},
   'GranuleUR': 'SMAP_L3_SSS_20161011_8DAYS_V5.0',
   'ProviderDates': [{'Type': 'Insert', 'Date': '2022-02-11T18:21:39.920Z'},
    {'Type': 'Update', 'Date': '2022-02-11T18:21:39.921Z'}],
   'SpatialExtent': {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangles': [{'WestBoundingCoordinate': -180,
        'SouthBoundingCoordinate': -90,
        'EastBoundingCoordinate': 180,
        'NorthB

In [13]:
granules_load = [] #list of granules to load (size of in situ time)
i=0
t=time[i]
start_date = str(t)
end_date = str(t+np.timedelta64(59+59*60+23*60*60, 's'))
date_range = (start_date, end_date) # this format is needed for 'requests'
date_ranges.add(date_range)

download_size = 0
GranuleQuery = DataGranules().parameters(
    concept_id=concept_id,
    bounding_box=bounding_box,
    temporal=t)
granules = GranuleQuery.get()
for granule in granules:
    # we use extend to aggregate the results in the same list.
    matching_granules.extend(granule.data_links(s3_only=True))
    download_size += granule.size()
print(f"date: {t} , total granules: {len(granules)}, size(MB): {round(download_size, 2)}")
print(granules)
    # for granule in granules:
    # url = granule['links'][0]['href']
    # id = granule['title'] + '.nc'
    # r = requests.get(url)
    # out_file = Path(output_dir, id)
    # granule_time=str(np.datetime64(granule['time_start'])+np.timedelta64(4, 'D'))
    # # only download the granule if granule time + 4 days is between start_date and end_date
    # if (granule_time >= start_date) & (granule_time <= end_date):
    #     print(url)
    #     print("date range: "+str(start_date)+'-'+str(end_date)+', file: '+str(id))
    #     granules_load.append(f'{output_dir}/{id}')
    #     # if the file doesn't exist locally, download
    #     if (not out_file.exists()):
    #         print('downloading', url, id)
    #         with open(out_file, 'wb') as f:
    #             f.write(r.content)    
    #     else:
    #         print('file already downloaded')  

2016-06-16T00:00:00.000000000/2016-06-16T23:59:59.000000000
The provided start date was not recognized


UnboundLocalError: local variable 'parsed_to' referenced before assignment

In [9]:
granules_load = [] #list of granules to load (size of in situ time)
for i in range(len(time)):
    t=time[i]
    start_date = str(t)
    end_date = str(t+np.timedelta64(59+59*60+23*60*60, 's'))
    date_range = start_date + "/" + end_date # this format is needed for 'requests'
    print(date_range)
    
    # Look for all the files in our date range/bounding box and get the urls - 8 days running mean so returns 9 files for 1 day
    response = requests.get(granule_url,
                           params={'concept_id': concept_id_sss,
                               'temporal': date_range,  
                               'bounding_box': bounding_box,
                               'page_size': 200},
                           headers={'Accept': 'application/json'})
    # print(response.headers['CMR-Hits'])
    granules = response.json()['feed']['entry']
    urls = []
    for granule in granules:
        urls.append(granule['links'][0]['href'])

    # Download the file for that specific day among all the files
    for granule in granules:
        url = granule['links'][0]['href']
        id = granule['title'] + '.nc'
        r = requests.get(url)
        out_file = Path(output_dir, id)
        granule_time=str(np.datetime64(granule['time_start'])+np.timedelta64(4, 'D'))
        # only download the granule if granule time + 4 days is between start_date and end_date
        if (granule_time >= start_date) & (granule_time <= end_date):
            print(url)
            print("date range: "+str(start_date)+'-'+str(end_date)+', file: '+str(id))
            granules_load.append(f'{output_dir}/{id}')
            # if the file doesn't exist locally, download
            if (not out_file.exists()):
                print('downloading', url, id)
                with open(out_file, 'wb') as f:
                    f.write(r.content)    
            else:
                print('file already downloaded')  

2016-06-16T00:00:00.000000000/2016-06-16T23:59:59.000000000


/tmp/ipykernel_14255/479041215.py:28: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  granule_time=str(np.datetime64(granule['time_start'])+np.timedelta64(4, 'D'))


https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-public/SMAP_JPL_L3_SSS_CAP_8DAY-RUNNINGMEAN_V5/2016/164/SMAP_L3_SSS_20160616_8DAYS_V5.0.nc.md5
date range: 2016-06-16T00:00:00.000000000-2016-06-16T23:59:59.000000000, file: SMAP_L3_SSS_20160616_8DAYS_V5.0.nc
file already downloaded
2016-06-16T00:00:00.000000000/2016-06-16T23:59:59.000000000
https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-public/SMAP_JPL_L3_SSS_CAP_8DAY-RUNNINGMEAN_V5/2016/164/SMAP_L3_SSS_20160616_8DAYS_V5.0.nc.md5
date range: 2016-06-16T00:00:00.000000000-2016-06-16T23:59:59.000000000, file: SMAP_L3_SSS_20160616_8DAYS_V5.0.nc
file already downloaded
2016-06-16T00:00:00.000000000/2016-06-16T23:59:59.000000000
https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-public/SMAP_JPL_L3_SSS_CAP_8DAY-RUNNINGMEAN_V5/2016/164/SMAP_L3_SSS_20160616_8DAYS_V5.0.nc.md5
date range: 2016-06-16T00:00:00.000000000-2016-06-16T23:59:59.000000000, file: SMAP_L3_SSS_20160616_8DAYS_V5.0.nc
file already downloade

In [11]:
urls

['https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/SMAP_JPL_L3_SSS_CAP_8DAY-RUNNINGMEAN_V5/2016/273/SMAP_L3_SSS_20161003_8DAYS_V5.0.nc',
 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/SMAP_JPL_L3_SSS_CAP_8DAY-RUNNINGMEAN_V5/2016/274/SMAP_L3_SSS_20161004_8DAYS_V5.0.nc',
 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-public/SMAP_JPL_L3_SSS_CAP_8DAY-RUNNINGMEAN_V5/2016/275/SMAP_L3_SSS_20161005_8DAYS_V5.0.nc.md5',
 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/SMAP_JPL_L3_SSS_CAP_8DAY-RUNNINGMEAN_V5/2016/276/SMAP_L3_SSS_20161006_8DAYS_V5.0.nc',
 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/SMAP_JPL_L3_SSS_CAP_8DAY-RUNNINGMEAN_V5/2016/277/SMAP_L3_SSS_20161007_8DAYS_V5.0.nc',
 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-public/SMAP_JPL_L3_SSS_CAP_8DAY-RUNNINGMEAN_V5/2016/278/SMAP_L3_SSS_20161008_8DAYS_V5.0.nc.md5',
 'https://archive.podaac.earthdata.nasa.gov/

In [10]:
# granules_load
# granules_load=['/home/jovyan/Data/SASSIE/collocation/satellite/smap_jpl_l3/SMAP_L3_SSS_20160616_8DAYS_V5.0.nc',
#  '/home/jovyan/Data/SASSIE/collocation/satellite/smap_jpl_l3/SMAP_L3_SSS_20160616_8DAYS_V5.0.nc',
#  '/home/jovyan/Data/SASSIE/collocation/satellite/smap_jpl_l3/SMAP_L3_SSS_20160616_8DAYS_V5.0.nc',
#  '/home/jovyan/Data/SASSIE/collocation/satellite/smap_jpl_l3/SMAP_L3_SSS_20160616_8DAYS_V5.0.nc',
#  '/home/jovyan/Data/SASSIE/collocation/satellite/smap_jpl_l3/SMAP_L3_SSS_20160714_8DAYS_V5.0.nc',
#  '/home/jovyan/Data/SASSIE/collocation/satellite/smap_jpl_l3/SMAP_L3_SSS_20160714_8DAYS_V5.0.nc',
#  '/home/jovyan/Data/SASSIE/collocation/satellite/smap_jpl_l3/SMAP_L3_SSS_20160714_8DAYS_V5.0.nc',
#  '/home/jovyan/Data/SASSIE/collocation/satellite/smap_jpl_l3/SMAP_L3_SSS_20160714_8DAYS_V5.0.nc',
#  '/home/jovyan/Data/SASSIE/collocation/satellite/smap_jpl_l3/SMAP_L3_SSS_20160714_8DAYS_V5.0.nc',
#  '/home/jovyan/Data/SASSIE/collocation/satellite/smap_jpl_l3/SMAP_L3_SSS_20160819_8DAYS_V5.0.nc',
#  '/home/jovyan/Data/SASSIE/collocation/satellite/smap_jpl_l3/SMAP_L3_SSS_20160819_8DAYS_V5.0.nc',
#  '/home/jovyan/Data/SASSIE/collocation/satellite/smap_jpl_l3/SMAP_L3_SSS_20160915_8DAYS_V5.0.nc',
#  '/home/jovyan/Data/SASSIE/collocation/satellite/smap_jpl_l3/SMAP_L3_SSS_20160915_8DAYS_V5.0.nc',
#  '/home/jovyan/Data/SASSIE/collocation/satellite/smap_jpl_l3/SMAP_L3_SSS_20160915_8DAYS_V5.0.nc',
#  '/home/jovyan/Data/SASSIE/collocation/satellite/smap_jpl_l3/SMAP_L3_SSS_20160915_8DAYS_V5.0.nc',
#  '/home/jovyan/Data/SASSIE/collocation/satellite/smap_jpl_l3/SMAP_L3_SSS_20160915_8DAYS_V5.0.nc',
#  '/home/jovyan/Data/SASSIE/collocation/satellite/smap_jpl_l3/SMAP_L3_SSS_20160915_8DAYS_V5.0.nc',
#  '/home/jovyan/Data/SASSIE/collocation/satellite/smap_jpl_l3/SMAP_L3_SSS_20160916_8DAYS_V5.0.nc',
#  '/home/jovyan/Data/SASSIE/collocation/satellite/smap_jpl_l3/SMAP_L3_SSS_20160916_8DAYS_V5.0.nc',
#  '/home/jovyan/Data/SASSIE/collocation/satellite/smap_jpl_l3/SMAP_L3_SSS_20160916_8DAYS_V5.0.nc',
#  '/home/jovyan/Data/SASSIE/collocation/satellite/smap_jpl_l3/SMAP_L3_SSS_20161007_8DAYS_V5.0.nc',
#  '/home/jovyan/Data/SASSIE/collocation/satellite/smap_jpl_l3/SMAP_L3_SSS_20161007_8DAYS_V5.0.nc',
#  '/home/jovyan/Data/SASSIE/collocation/satellite/smap_jpl_l3/SMAP_L3_SSS_20161007_8DAYS_V5.0.nc',
#  '/home/jovyan/Data/SASSIE/collocation/satellite/smap_jpl_l3/SMAP_L3_SSS_20161007_8DAYS_V5.0.nc',
#  '/home/jovyan/Data/SASSIE/collocation/satellite/smap_jpl_l3/SMAP_L3_SSS_20161007_8DAYS_V5.0.nc',
#  '/home/jovyan/Data/SASSIE/collocation/satellite/smap_jpl_l3/SMAP_L3_SSS_20161007_8DAYS_V5.0.nc']

In [36]:
ds_smap_L3 = xr.open_mfdataset(
    paths=granules_load[-9:],
    combine='nested',
    concat_dim='time',
    decode_cf=True,
    coords='minimal',
    chunks={'time': 1}  
    ).sel(longitude=slice(lonrange[0],lonrange[1]), latitude=slice(latrange[1],latrange[0]))
ds_smap_L3

<xarray.Dataset>
Dimensions:               (time: 9, latitude: 48, longitude: 120)
Coordinates:
  * latitude              (latitude) float32 79.88 79.62 79.38 ... 68.38 68.12
  * longitude             (longitude) float32 -159.9 -159.6 ... -130.4 -130.1
  * time                  (time) datetime64[ns] 2016-09-16T12:00:00 ... 2016-...
Data variables:
    smap_sss              (time, latitude, longitude) float32 dask.array<chunksize=(1, 48, 120), meta=np.ndarray>
    anc_sss               (time, latitude, longitude) float32 dask.array<chunksize=(1, 48, 120), meta=np.ndarray>
    anc_sst               (time, latitude, longitude) float32 dask.array<chunksize=(1, 48, 120), meta=np.ndarray>
    smap_spd              (time, latitude, longitude) float32 dask.array<chunksize=(1, 48, 120), meta=np.ndarray>
    smap_high_spd         (time, latitude, longitude) float32 dask.array<chunksize=(1, 48, 120), meta=np.ndarray>
    weight                (time, latitude, longitude) float32 dask.array<chunksize=(1, 48, 120), meta=np.ndarray>
    land_fraction         (time, latitude, longitude) float32 dask.array<chunksize=(1, 48, 120), meta=np.ndarray>
    ice_fraction          (time, latitude, longitude) float32 dask.array<chunksize=(1, 48, 120), meta=np.ndarray>
    smap_sss_uncertainty  (time, latitude, longitude) float32 dask.array<chunksize=(1, 48, 120), meta=np.ndarray>
Attributes: (12/40)
    title:                       SMAP 0.25x0.25 deg grid averaged 8-day SSS/WSPD
    institution:                 Jet Propulsion Laboratory
    source:                      SMAP L2B SSS
    history:                     DATA_SOURCE_VERSION V5.0 L2B SMAP SSS/WSPD
    comment:                     Gaussian-weighted map gridding of SMAP L2B S...
    Gaussian_window_radius:      45.0
    ...                          ...
    creator_name:                JPL
    creator_email:               fore@jpl.nasa.gov
    publisher_name:              Alexander G. Fore
    publisher_email:             fore@jpl.nasa.gov
    contributor_name:            Alexander Fore, Simon Yueh, Wenqing Tang, Ak...
    references:                  10.1109/TGRS.2016.2601486, 10.1109/TGRS.2016...

In [46]:
print(insitu.y[-9:].values, insitu.x[-9:].values, insitu.t[-9:].values)
print(ds.latitude[-9:].values, ds.longitude[-9:].values, ds.time[-9:].values)

[75.015      75.44333333 75.775      72.         73.005      74.00666667
 74.98333333 76.01666667 77.05333333] [-157.46       -160.54166667 -163.80333333 -150.         -149.99666667
 -150.         -150.00666667 -149.995      -149.98666667] ['2016-09-15T18:39:00.000000000' '2016-09-15T19:03:00.000000000'
 '2016-09-15T19:26:00.000000000' '2016-10-06T20:36:00.000000000'
 '2016-10-06T20:59:00.000000000' '2016-10-06T21:22:00.000000000'
 '2016-10-06T21:45:00.000000000' '2016-10-06T22:14:00.000000000'
 '2016-10-06T22:39:00.000000000']
[75.015      75.44333333 75.775      72.         73.005      74.00666667
 74.98333333 76.01666667 77.05333333] [-157.46       -160.54166667 -163.80333333 -150.         -149.99666667
 -150.         -150.00666667 -149.995      -149.98666667] ['2016-09-16T12:00:00.000000000' '2016-09-16T12:00:00.000000000'
 '2016-09-16T12:00:00.000000000' '2016-10-07T12:00:00.000000000'
 '2016-10-07T12:00:00.000000000' '2016-10-07T12:00:00.000000000'
 '2016-10-07T12:00:00.000000000

In [51]:
ds=ds_smap_L3.smap_sss.interp(longitude=('z', insitu.x[-9:].values), latitude=('z', insitu.y[-9:].values), time=('z', insitu.t[-9:].values))
ds

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [50]:
# ds=ds_smap_L3.smap_sss.interp(latitude=insitu.y[-9:], longitude=insitu.x[-9:], method='nearest')
ds=ds_smap_L3.smap_sss.interp(longitude=('z', insitu.x[-9:]), latitude=('z', insitu.y[-9:]), time=('z', insitu.t[-9:]), method='nearest')
ds

TypeError: Using a DataArray object to construct a variable is ambiguous, please extract the data using the .data property.

In [49]:
ds_smap_L3

<xarray.Dataset>
Dimensions:               (time: 9, latitude: 48, longitude: 120)
Coordinates:
  * latitude              (latitude) float32 79.88 79.62 79.38 ... 68.38 68.12
  * longitude             (longitude) float32 -159.9 -159.6 ... -130.4 -130.1
  * time                  (time) datetime64[ns] 2016-09-16T12:00:00 ... 2016-...
Data variables:
    smap_sss              (time, latitude, longitude) float32 dask.array<chunksize=(1, 48, 120), meta=np.ndarray>
    anc_sss               (time, latitude, longitude) float32 dask.array<chunksize=(1, 48, 120), meta=np.ndarray>
    anc_sst               (time, latitude, longitude) float32 dask.array<chunksize=(1, 48, 120), meta=np.ndarray>
    smap_spd              (time, latitude, longitude) float32 dask.array<chunksize=(1, 48, 120), meta=np.ndarray>
    smap_high_spd         (time, latitude, longitude) float32 dask.array<chunksize=(1, 48, 120), meta=np.ndarray>
    weight                (time, latitude, longitude) float32 dask.array<chunksize=(1, 48, 120), meta=np.ndarray>
    land_fraction         (time, latitude, longitude) float32 dask.array<chunksize=(1, 48, 120), meta=np.ndarray>
    ice_fraction          (time, latitude, longitude) float32 dask.array<chunksize=(1, 48, 120), meta=np.ndarray>
    smap_sss_uncertainty  (time, latitude, longitude) float32 dask.array<chunksize=(1, 48, 120), meta=np.ndarray>
Attributes: (12/40)
    title:                       SMAP 0.25x0.25 deg grid averaged 8-day SSS/WSPD
    institution:                 Jet Propulsion Laboratory
    source:                      SMAP L2B SSS
    history:                     DATA_SOURCE_VERSION V5.0 L2B SMAP SSS/WSPD
    comment:                     Gaussian-weighted map gridding of SMAP L2B S...
    Gaussian_window_radius:      45.0
    ...                          ...
    creator_name:                JPL
    creator_email:               fore@jpl.nasa.gov
    publisher_name:              Alexander G. Fore
    publisher_email:             fore@jpl.nasa.gov
    contributor_name:            Alexander Fore, Simon Yueh, Wenqing Tang, Ak...
    references:                  10.1109/TGRS.2016.2601486, 10.1109/TGRS.2016...

In [ ]:
# interp = RegularGridInterpolator( (np.array(ds_smap_L3.latitude),np.array(ds_smap_L3.longitude)), np.array(ds_smap_L3.smap_sss), method='nearest') 
# y=np.array(insitu.y.values[0])
# x=np.array(insitu.x.values[0])

# res = interp([y,x])
# res
# # a=insitu.x[i].values
# # a.shape
# # insitu.y[i].values
# # ds_smap_L3.latitude.values
# # ds_smap_L3.longitude.values

In [ ]:
# print(y,x,lat,lon)
# print(data[14:17,40:43])

In [12]:
# Opening the file in the cloud, not downloading it
# #returns nothing for SMAP JPL
# paths=[fs.open(f) for f in urls]
SSS_files = fs.glob(join("podaac-ops-cumulus-protected/", ShortName_sss, '*.nc'))
# fileset = fs.open(urls[0])
# ds_stacked = xr.open_mfdataset(fileset, combine='by_coords')
# SSS = xr.open_mfdataset(
#     paths=paths,
#     combine='by_coord',
#     decode_cf=True,
#     coords='minimal',
#     chunks={'time': 1}  
#     )
SSS_files

[]